### Visual Genome Process

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-Visual Genome Process"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def lower_strip_func(x):
    try:
        var_low = str(x).lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [5]:
def list_to_text(text_list):
    for x in text_list:
        x = " , ".join(str(x))
    return x

In [6]:
test_list = ["ali", "ata"]

In [7]:
list_to_text(test_list)

'a , t , a'

#### Visual Genome Data

In [8]:
df_genome_objects = pd.read_json(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Word Gnome/objects.json")
df_genome_objects

,image_id,objects,image_url,merged_object_ids
0,1,"[{'synsets': ['tree.n.01'], 'h': 557, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
1,2,"[{'synsets': ['road.n.01'], 'h': 254, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
2,3,"[{'synsets': ['booth.n.02'], 'h': 389, 'object...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
3,4,"[{'synsets': ['floor.n.01'], 'h': 168, 'object...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
4,5,"[{'synsets': ['room.n.01'], 'h': 599, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
...,...,...,...,...
108072,2417992,"[{'synsets': [], 'h': 111, 'object_id': 378678...",http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN
108073,2417993,"[{'synsets': ['beach.n.01'], 'h': 194, 'object...",http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN
108074,2417994,"[{'synsets': ['bell.n.01'], 'h': 14, 'object_i...",http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN
108075,2417995,"[{'synsets': ['barroom.n.01'], 'h': 29, 'objec...",http://crowdfile.blob.core.chinacloudapi.cn/46...,NaN


In [9]:
df_genome_attributes = pd.read_json(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Word Gnome/attributes.json")
df_genome_attributes

,image_id,attributes
0,1,"[{'synsets': ['clock.n.01'], 'h': 339, 'object..."
1,2,"[{'synsets': [], 'h': 103, 'object_id': 5069, ..."
2,3,"[{'synsets': [], 'h': 79, 'object_id': 5091, '..."
3,4,"[{'synsets': ['curtain.n.01'], 'h': 300, 'obje..."
4,5,"[{'synsets': ['floor.n.01'], 'h': 108, 'object..."
...,...,...
108072,2417992,"[{'synsets': ['train.n.01'], 'h': 322, 'object..."
108073,2417993,"[{'synsets': ['foot.n.01'], 'h': 84, 'object_i..."
108074,2417994,"[{'synsets': ['sheep.n.01'], 'h': 137, 'object..."
108075,2417995,"[{'synsets': ['window.n.01'], 'h': 50, 'object..."


In [10]:
df_genome_relationships = pd.read_json(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Word Gnome/relationships.json")
df_genome_relationships

,relationships,image_id
0,"[{'predicate': 'ON', 'object': {'h': 290, 'obj...",1
1,"[{'predicate': 'wears', 'object': {'name': 'ba...",2
2,"[{'predicate': 'in front of', 'object': {'name...",3
3,"[{'predicate': 'has', 'object': {'name': 'padd...",4
4,"[{'predicate': 'ON', 'object': {'name': 'floor...",5
...,...,...
108072,"[{'predicate': 'on top of', 'object': {'name':...",2417992
108073,"[{'predicate': 'has a', 'object': {'name': 'gu...",2417993
108074,"[{'predicate': 'standing in', 'object': {'name...",2417994
108075,"[{'predicate': 'ON', 'object': {'name': 'train...",2417995


In [11]:
#df_genome_objects.loc[0,"objects"]

In [12]:
#df_genome_attributes.loc[0,"attributes"]

In [13]:
#df_genome_relationships.loc[0,"relationships"]

##### Object Data Analysis

In [14]:
#object_data = zip(df_genome_objects["image_id"],df_genome_objects["objects"],df_genome_objects["image_url"])

In [15]:
## First Way Too Slow
#df_object_result = pd.DataFrame()
#for image_id, objects, image_url in object_data:
#    image_id_list = []
#    names_list = []
#    object_id_list = []
#    synsets_list = []
#    height_list = []
#    width_list = []
#    x_koor_list = []
#    y_koor_list = []
#    image_url_list = []
#    for dict in objects:
#        names = dict["names"]
#        object_id = dict["object_id"]
#        synsets = dict["synsets"]
#        height = dict["h"]
#        width = dict["w"]
#        x_koor = dict["x"]
#        y_koor = dict["y"]
#        image_id_list.append(int(image_id))
#        names_list.append(names)
#        object_id_list.append(int(object_id))
#        synsets_list.append(synsets)
#        height_list.append(int(height))
#        width_list.append(int(width))
#        x_koor_list.append(int(x_koor))
#        y_koor_list.append(int(y_koor))
#        image_url_list.append(image_url)
#    df_var = pd.DataFrame({"image_id":image_id_list,"names":names_list,"object_id":object_id_list,"synsets":synsets_list,"height":height_list,"width":width_list,"x_koor":x_koor_list,"y_koor":y_koor_list,"image_url":image_url_list})
#    df_object_result = pd.concat([df_object_result,df_var], axis=0)
#    
#df_object_result.reset_index(drop=True, inplace=True)
#df_object_result

In [16]:
#df_object_result.to_csv("Object_First_Result.csv", index=False)

In [17]:
object_data = zip(df_genome_objects["image_id"],df_genome_objects["objects"],df_genome_objects["image_url"])

In [18]:
# Second Way Too Fast
image_id_list = []
names_list = []
object_id_list = []
synsets_list = []
merged_object_list = []
height_list = []
width_list = []
x_koor_list = []
y_koor_list = []
image_url_list = []
for image_id, objects, image_url in object_data:
    for dict in objects:
        names = dict["names"]
        object_id = dict["object_id"]
        synsets = dict["synsets"]
        try:
            merged_object = dict["merged_object_ids"]
        except:
            merged_object = []
        height = dict["h"]
        width = dict["w"]
        x_koor = dict["x"]
        y_koor = dict["y"]
        image_id_list.append(int(image_id))
        names_list.append(names)
        object_id_list.append(int(object_id))
        synsets_list.append(synsets)
        merged_object_list.append(merged_object)
        height_list.append(int(height))
        width_list.append(int(width))
        x_koor_list.append(int(x_koor))
        y_koor_list.append(int(y_koor))
        image_url_list.append(image_url)
    
df_object_result = pd.DataFrame({"object_id":object_id_list,"names":names_list,"image_id":image_id_list,"image_url":image_url_list,"synsets":synsets_list,"merged_object_ids":merged_object_list,"height":height_list,"width":width_list,"x_koor":x_koor_list,"y_koor":y_koor_list})
df_object_result   

,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,[trees],1,https://cs.stanford.edu/people/rak248/VG_100K_...,[tree.n.01],[],557,799,0,0
1,1058534,[sidewalk],1,https://cs.stanford.edu/people/rak248/VG_100K_...,[sidewalk.n.01],[5046],290,722,78,308
2,1058508,[building],1,https://cs.stanford.edu/people/rak248/VG_100K_...,[building.n.01],[],538,222,1,0
3,1058539,[street],1,https://cs.stanford.edu/people/rak248/VG_100K_...,[street.n.01],[3798578],258,359,439,283
4,1058543,[wall],1,https://cs.stanford.edu/people/rak248/VG_100K_...,[wall.n.01],[],535,135,0,1
...,...,...,...,...,...,...,...,...,...,...
2516934,3506171,[number],2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,[numeral.n.01],[],24,28,188,228
2516935,3786799,[plate],2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,[plate.n.04],[],54,89,238,233
2516936,3103226,[road],2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,[road.n.01],[],83,90,312,247
2516937,3094589,[road],2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,[road.n.01],[],121,495,2,208


In [19]:
df_object_result["object_id"].nunique()

2516939

In [20]:
df_object_result["image_id"].nunique()

106284

In [21]:
df_object_result["names"] = [' , '.join(map(str, l)) for l in df_object_result['names']]
df_object_result["synsets"] = [' , '.join(map(str, l)) for l in df_object_result['synsets']]
df_object_result["merged_object_ids"] = [' , '.join(map(str, l)) for l in df_object_result['merged_object_ids']]
df_object_result

,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,trees,1,https://cs.stanford.edu/people/rak248/VG_100K_...,tree.n.01,,557,799,0,0
1,1058534,sidewalk,1,https://cs.stanford.edu/people/rak248/VG_100K_...,sidewalk.n.01,5046,290,722,78,308
2,1058508,building,1,https://cs.stanford.edu/people/rak248/VG_100K_...,building.n.01,,538,222,1,0
3,1058539,street,1,https://cs.stanford.edu/people/rak248/VG_100K_...,street.n.01,3798578,258,359,439,283
4,1058543,wall,1,https://cs.stanford.edu/people/rak248/VG_100K_...,wall.n.01,,535,135,0,1
...,...,...,...,...,...,...,...,...,...,...
2516934,3506171,number,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,numeral.n.01,,24,28,188,228
2516935,3786799,plate,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,plate.n.04,,54,89,238,233
2516936,3103226,road,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,road.n.01,,83,90,312,247
2516937,3094589,road,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,road.n.01,,121,495,2,208


In [22]:
df_object_result["names"] = df_object_result["names"].apply(lambda x: lower_strip_func(x))
df_object_result["synsets"] = df_object_result["synsets"].apply(lambda x: lower_strip_func(x))
df_object_result

,object_id,names,image_id,image_url,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,trees,1,https://cs.stanford.edu/people/rak248/VG_100K_...,tree.n.01,,557,799,0,0
1,1058534,sidewalk,1,https://cs.stanford.edu/people/rak248/VG_100K_...,sidewalk.n.01,5046,290,722,78,308
2,1058508,building,1,https://cs.stanford.edu/people/rak248/VG_100K_...,building.n.01,,538,222,1,0
3,1058539,street,1,https://cs.stanford.edu/people/rak248/VG_100K_...,street.n.01,3798578,258,359,439,283
4,1058543,wall,1,https://cs.stanford.edu/people/rak248/VG_100K_...,wall.n.01,,535,135,0,1
...,...,...,...,...,...,...,...,...,...,...
2516934,3506171,number,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,numeral.n.01,,24,28,188,228
2516935,3786799,plate,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,plate.n.04,,54,89,238,233
2516936,3103226,road,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,road.n.01,,83,90,312,247
2516937,3094589,road,2417997,http://crowdfile.blob.core.chinacloudapi.cn/46...,road.n.01,,121,495,2,208


In [23]:
#df_object_result_sample = df_object_result.head(100)
#df_object_result_sample.to_excel("Objects_Sample.xlsx", index=False)

In [24]:
df_object_result.to_csv("Visual_Genome_Objects_Analysis.csv", index=False)

##### Attributes Data Analysis

In [25]:
attribute_data = zip(df_genome_attributes["image_id"],df_genome_attributes["attributes"])

In [26]:
image_id_list = []
names_list = []
object_id_list = []
synsets_list = []
attributes_list = []
height_list = []
width_list = []
x_koor_list = []
y_koor_list = []
for image_id, attributes in attribute_data:
    for dict in attributes:
        names = dict["names"]
        object_id = dict["object_id"]
        synsets = dict["synsets"]
        try:
            attribute_var = dict["attributes"]
        except:
            attribute_var = []
        height = dict["h"]
        width = dict["w"]
        x_koor = dict["x"]
        y_koor = dict["y"]
        image_id_list.append(int(image_id))
        names_list.append(names)
        object_id_list.append(int(object_id))
        synsets_list.append(synsets)
        attributes_list.append(attribute_var)
        height_list.append(int(height))
        width_list.append(int(width))
        x_koor_list.append(int(x_koor))
        y_koor_list.append(int(y_koor))
    
df_attributes_result = pd.DataFrame({"object_id":object_id_list,"names":names_list,"image_id":image_id_list,"attributes":attributes_list,"synsets":synsets_list,"height":height_list,"width":width_list,"x_koor":x_koor_list,"y_koor":y_koor_list})
df_attributes_result 

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,[clock],1,"[green, tall]",[clock.n.01],339,79,421,91
1,5046,[street],1,[sidewalk],[street.n.01],262,714,77,328
2,5045,[shade],1,[],[shade.n.01],192,274,119,338
3,1058529,[man],1,[],[man.n.01],262,60,238,249
4,5048,[sneakers],1,[grey],[gym_shoe.n.01],26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,[sky],2417997,"[crystal clear, blue]",[sky.n.01],170,497,1,2
3802370,3103226,[road],2417997,[],[road.n.01],83,90,312,247
3802371,2712437,[doors],2417997,[],[door.n.01],144,50,98,127
3802372,3140197,[bus],2417997,[green],[bus.n.01],155,225,29,109


In [27]:
df_attributes_result["object_id"].nunique()

3802374

In [28]:
df_attributes_result["image_id"].nunique()

107194

In [29]:
df_attributes_result["names"] = [' , '.join(map(str, l)) for l in df_attributes_result['names']]
df_attributes_result["synsets"] = [' , '.join(map(str, l)) for l in df_attributes_result['synsets']]
df_attributes_result["attributes"] = [' , '.join(map(str, l)) for l in df_attributes_result['attributes']]
df_attributes_result

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,clock,1,"green , tall",clock.n.01,339,79,421,91
1,5046,street,1,sidewalk,street.n.01,262,714,77,328
2,5045,shade,1,,shade.n.01,192,274,119,338
3,1058529,man,1,,man.n.01,262,60,238,249
4,5048,sneakers,1,grey,gym_shoe.n.01,26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,sky,2417997,"crystal clear , blue",sky.n.01,170,497,1,2
3802370,3103226,road,2417997,,road.n.01,83,90,312,247
3802371,2712437,doors,2417997,,door.n.01,144,50,98,127
3802372,3140197,bus,2417997,green,bus.n.01,155,225,29,109


In [30]:
df_attributes_result["names"] = df_attributes_result["names"].apply(lambda x: lower_strip_func(x))
df_attributes_result["attributes"] = df_attributes_result["attributes"].apply(lambda x: lower_strip_func(x))
df_attributes_result["synsets"] = df_attributes_result["synsets"].apply(lambda x: lower_strip_func(x))
df_attributes_result

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,clock,1,"green , tall",clock.n.01,339,79,421,91
1,5046,street,1,sidewalk,street.n.01,262,714,77,328
2,5045,shade,1,,shade.n.01,192,274,119,338
3,1058529,man,1,,man.n.01,262,60,238,249
4,5048,sneakers,1,grey,gym_shoe.n.01,26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,sky,2417997,"crystal clear , blue",sky.n.01,170,497,1,2
3802370,3103226,road,2417997,,road.n.01,83,90,312,247
3802371,2712437,doors,2417997,,door.n.01,144,50,98,127
3802372,3140197,bus,2417997,green,bus.n.01,155,225,29,109


In [31]:
#df_attributes_result_sample = df_attributes_result.head(100)
#df_attributes_result_sample.to_excel("Attributes_Sample.xlsx", index=False)

In [32]:
df_attributes_result.to_csv("Visual_Genome_Attributes_Analysis.csv", index=False)

##### Relationships Data Analysis

In [33]:
relationships_data = zip(df_genome_relationships["image_id"],df_genome_relationships["relationships"])

In [34]:
df_genome_relationships.loc[0,"relationships"]

[{'predicate': 'ON',
  'object': {'h': 290,
   'object_id': 1058534,
   'merged_object_ids': [5046],
   'synsets': ['sidewalk.n.01'],
   'w': 722,
   'y': 308,
   'x': 78,
   'names': ['sidewalk']},
  'relationship_id': 15927,
  'synsets': ['along.r.01'],
  'subject': {'name': 'shade',
   'h': 192,
   'synsets': ['shade.n.01'],
   'object_id': 5045,
   'w': 274,
   'y': 338,
   'x': 119}},
 {'predicate': 'wears',
  'object': {'h': 28,
   'object_id': 1058525,
   'merged_object_ids': [5048],
   'synsets': ['shoe.n.01'],
   'w': 48,
   'y': 485,
   'x': 388,
   'names': ['shoes']},
  'relationship_id': 15928,
  'synsets': ['wear.v.01'],
  'subject': {'name': 'man',
   'h': 262,
   'synsets': ['man.n.01'],
   'object_id': 1058529,
   'w': 60,
   'y': 249,
   'x': 238}},
 {'predicate': 'has',
  'object': {'name': 'headlight',
   'h': 15,
   'synsets': ['headlight.n.01'],
   'object_id': 5050,
   'w': 23,
   'y': 366,
   'x': 514},
  'relationship_id': 15929,
  'synsets': ['have.v.01'],
  '

In [35]:
df_genome_relationships.loc[0,"relationships"][0]

{'predicate': 'ON',
 'object': {'h': 290,
  'object_id': 1058534,
  'merged_object_ids': [5046],
  'synsets': ['sidewalk.n.01'],
  'w': 722,
  'y': 308,
  'x': 78,
  'names': ['sidewalk']},
 'relationship_id': 15927,
 'synsets': ['along.r.01'],
 'subject': {'name': 'shade',
  'h': 192,
  'synsets': ['shade.n.01'],
  'object_id': 5045,
  'w': 274,
  'y': 338,
  'x': 119}}

In [36]:
df_genome_relationships.loc[0,"relationships"][0]["object"]["names"]

['sidewalk']

In [37]:
image_id_list = []
obj_names_list = []
obj_object_id_list = []
obj_synsets_list = []
obj_merged_object_list = []
obj_height_list = []
obj_width_list = []
obj_x_koor_list = []
obj_y_koor_list = []
relationship_id_list = []
synsets_list = []
predicate_list = []
sub_name_list = []
sub_object_id_list = []
sub_synsets_list = []
sub_height_list = []
sub_width_list = []
sub_x_koor_list = []
sub_y_koor_list = []
for image_id, relationships in relationships_data:
    for dict in relationships:
        try:
            obj_names = dict["object"]["names"]
        except:
            obj_names = []
        obj_object_id = dict["object"]["object_id"]
        obj_synsets = dict["object"]["synsets"]
        try:
            obj_merged_object = dict["object"]["merged_object_ids"]
        except:
            obj_merged_object = []
        obj_height = dict["object"]["h"]
        obj_width = dict["object"]["w"]
        obj_x_koor = dict["object"]["x"]
        obj_y_koor = dict["object"]["y"]
        relationship_id = dict["relationship_id"]
        synsets = dict["synsets"]
        predicate = dict["predicate"]
        try:
            sub_names = dict["subject"]["name"]
        except:
            sub_names = ""
        sub_object_id = dict["subject"]["object_id"]
        sub_synsets = dict["subject"]["synsets"]
        sub_height = dict["subject"]["h"]
        sub_width = dict["subject"]["w"]
        sub_x_koor = dict["subject"]["x"]
        sub_y_koor = dict["subject"]["y"]

        image_id_list.append(int(image_id))
        obj_names_list.append(obj_names)
        obj_object_id_list.append(int(obj_object_id))
        obj_synsets_list.append(obj_synsets)
        obj_merged_object_list.append(obj_merged_object)
        obj_height_list.append(int(obj_height))
        obj_width_list.append(int(obj_width))
        obj_x_koor_list.append(int(obj_x_koor))
        obj_y_koor_list.append(int(obj_y_koor))
        relationship_id_list.append(relationship_id)
        synsets_list.append(synsets)
        predicate_list.append(predicate)
        sub_name_list.append(sub_names)
        sub_object_id_list.append(int(sub_object_id))
        sub_synsets_list.append(sub_synsets)
        sub_height_list.append(int(sub_height))
        sub_width_list.append(int(sub_width))
        sub_x_koor_list.append(int(sub_x_koor))
        sub_y_koor_list.append(int(sub_y_koor))
    
df_relationships_result = pd.DataFrame({"image_id":image_id_list,"obj_names":obj_names_list,"obj_object_id":obj_object_id_list,"obj_synsets":obj_synsets_list,
"obj_merged_object_ids":obj_merged_object_list,"obj_height":obj_height_list,"obj_width":obj_width_list,"obj_x_koor":obj_x_koor_list,"obj_y_koor":obj_y_koor_list,
"relationships_id":relationship_id_list,"synsets":synsets_list,"sub_name":sub_name_list,"sub_object_id":sub_object_id_list,"sub_synsets":sub_synsets_list,
"sub_height":sub_height_list,"sub_width":sub_width_list,"sub_x_koor":sub_x_koor_list,"sub_y_koor":sub_y_koor_list,"predicate":predicate_list})
df_relationships_result 

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,[sidewalk],1058534,[sidewalk.n.01],[5046],290,722,78,308,15927,[along.r.01],shade,5045,[shade.n.01],192,274,119,338,ON
1,1,[shoes],1058525,[shoe.n.01],[5048],28,48,388,485,15928,[wear.v.01],man,1058529,[man.n.01],262,60,238,249,wears
2,1,[],5050,[headlight.n.01],[],15,23,514,366,15929,[have.v.01],car,5049,[car.n.01],98,74,479,315,has
3,1,[],1058508,[building.n.01],[],536,218,1,2,15930,[along.r.01],sign,1058507,[sign.n.02],182,88,118,13,ON
4,1,[],1058534,[sidewalk.n.01],[],266,722,77,331,15931,[along.r.01],tree trunk,5055,[trunk.n.01],327,87,622,234,ON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,[],3103225,[bus.n.01],[],193,289,14,110,4245033,[along.r.01],plate,3786799,[plate.n.04],54,89,238,233,ON
2316100,2417997,[],3786800,[desert.n.01],[],119,157,301,211,4245034,[in.r.01],,3103225,[bus.n.01],182,287,14,110,IN
2316101,2417997,[],3103226,[road.n.01],[],83,90,312,247,3722727,[along.r.01],bus,3103225,[bus.n.01],193,289,14,110,ON
2316102,2417997,[],2712437,[door.n.01],[],144,50,98,127,3880229,[],bus,3103225,[bus.n.01],193,289,14,110,with


In [38]:
df_relationships_result["obj_object_id"].nunique()

1300368

In [39]:
df_relationships_result["sub_object_id"].nunique() 

1223263

In [40]:
df_relationships_result["obj_names"] = [' , '.join(map(str, l)) for l in df_relationships_result['obj_names']]
df_relationships_result["obj_synsets"] = [' , '.join(map(str, l)) for l in df_relationships_result['obj_synsets']]
df_relationships_result["synsets"] = [' , '.join(map(str, l)) for l in df_relationships_result['synsets']]
df_relationships_result["sub_synsets"] = [' , '.join(map(str, l)) for l in df_relationships_result['sub_synsets']]
df_relationships_result["obj_merged_object_ids"] = [' , '.join(map(str, l)) for l in df_relationships_result['obj_merged_object_ids']]
df_relationships_result

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,sidewalk,1058534,sidewalk.n.01,5046,290,722,78,308,15927,along.r.01,shade,5045,shade.n.01,192,274,119,338,ON
1,1,shoes,1058525,shoe.n.01,5048,28,48,388,485,15928,wear.v.01,man,1058529,man.n.01,262,60,238,249,wears
2,1,,5050,headlight.n.01,,15,23,514,366,15929,have.v.01,car,5049,car.n.01,98,74,479,315,has
3,1,,1058508,building.n.01,,536,218,1,2,15930,along.r.01,sign,1058507,sign.n.02,182,88,118,13,ON
4,1,,1058534,sidewalk.n.01,,266,722,77,331,15931,along.r.01,tree trunk,5055,trunk.n.01,327,87,622,234,ON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,,3103225,bus.n.01,,193,289,14,110,4245033,along.r.01,plate,3786799,plate.n.04,54,89,238,233,ON
2316100,2417997,,3786800,desert.n.01,,119,157,301,211,4245034,in.r.01,,3103225,bus.n.01,182,287,14,110,IN
2316101,2417997,,3103226,road.n.01,,83,90,312,247,3722727,along.r.01,bus,3103225,bus.n.01,193,289,14,110,ON
2316102,2417997,,2712437,door.n.01,,144,50,98,127,3880229,,bus,3103225,bus.n.01,193,289,14,110,with


In [41]:
df_relationships_result["obj_names"] = df_relationships_result["obj_names"].apply(lambda x: lower_strip_func(x))
df_relationships_result["obj_synsets"] = df_relationships_result["obj_synsets"].apply(lambda x: lower_strip_func(x))
df_relationships_result["synsets"] = df_relationships_result["synsets"].apply(lambda x: lower_strip_func(x))
df_relationships_result["sub_name"] = df_relationships_result["sub_name"].apply(lambda x: lower_strip_func(x))
df_relationships_result["sub_synsets"] = df_relationships_result["sub_synsets"].apply(lambda x: lower_strip_func(x))
df_relationships_result["predicate"] = df_relationships_result["predicate"].apply(lambda x: lower_strip_func(x))
df_relationships_result

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,sidewalk,1058534,sidewalk.n.01,5046,290,722,78,308,15927,along.r.01,shade,5045,shade.n.01,192,274,119,338,on
1,1,shoes,1058525,shoe.n.01,5048,28,48,388,485,15928,wear.v.01,man,1058529,man.n.01,262,60,238,249,wears
2,1,,5050,headlight.n.01,,15,23,514,366,15929,have.v.01,car,5049,car.n.01,98,74,479,315,has
3,1,,1058508,building.n.01,,536,218,1,2,15930,along.r.01,sign,1058507,sign.n.02,182,88,118,13,on
4,1,,1058534,sidewalk.n.01,,266,722,77,331,15931,along.r.01,tree trunk,5055,trunk.n.01,327,87,622,234,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,,3103225,bus.n.01,,193,289,14,110,4245033,along.r.01,plate,3786799,plate.n.04,54,89,238,233,on
2316100,2417997,,3786800,desert.n.01,,119,157,301,211,4245034,in.r.01,,3103225,bus.n.01,182,287,14,110,in
2316101,2417997,,3103226,road.n.01,,83,90,312,247,3722727,along.r.01,bus,3103225,bus.n.01,193,289,14,110,on
2316102,2417997,,2712437,door.n.01,,144,50,98,127,3880229,,bus,3103225,bus.n.01,193,289,14,110,with


In [42]:
#df_relationships_result_sample = df_relationships_result.head(100)
#df_relationships_result_sample.to_excel("Relationships_Sample.xlsx", index=False)

In [43]:
df_relationships_result.to_csv("Visual_Genome_Relationships_Analysis.csv", index=False)

#### Copy Move And Delete

In [44]:
output_file = glob.glob(f"Visual_Genome_*_Analysis.csv")
output_file

['Word_Gnome_Objects_Analysis.csv',
 'Word_Gnome_Attributes_Analysis.csv',
 'Word_Gnome_Relationships_Analysis.csv']

In [45]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [46]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass